In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory('dataset/data_pumpkin/dtrain',
                                             target_size=(64, 64),
                                             batch_size=32,
                                             class_mode='categorical')

test_set = test_datagen.flow_from_directory('dataset/data_pumpkin/dtest',
                                           target_size=(64, 64),
                                           batch_size=32,
                                           class_mode='categorical')

Found 1490 images belonging to 15 classes.
Found 1490 images belonging to 15 classes.


In [3]:
print(test_set.class_indices)

{'Alternaria cucumerina': 0, 'Alternaria leaf blight': 1, 'Aphids': 2, 'Armyworms': 3, 'Bacterial leaf spot': 4, 'Bacterial wilt': 5, 'Cucumber beetles': 6, 'Flea beetles': 7, 'Fusarium': 8, 'Gummy stem blight': 9, 'Phytophthora bligh': 10, 'Squash bug': 11, 'Squash vine borer': 12, 'Thrips (Western flower thrips)': 13, 'healthy_pumpkin': 14}


In [3]:

# Build the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=15, activation='softmax'))  # Assuming 10 classes

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
model.fit(
    train_set,
    epochs=25,
    validation_data=test_set
)

Epoch 1/25
47/47 [==============================] - 31s 660ms/step - loss: 1.9301 - accuracy: 0.3510 - val_loss: 1.9131 - val_accuracy: 0.3510
Epoch 2/25
47/47 [==============================] - 28s 595ms/step - loss: 1.8587 - accuracy: 0.3953 - val_loss: 1.8896 - val_accuracy: 0.3866
Epoch 3/25
47/47 [==============================] - 27s 585ms/step - loss: 1.8253 - accuracy: 0.3846 - val_loss: 1.7588 - val_accuracy: 0.4114
Epoch 4/25
47/47 [==============================] - 28s 601ms/step - loss: 1.7174 - accuracy: 0.4275 - val_loss: 1.7063 - val_accuracy: 0.4430
Epoch 5/25
47/47 [==============================] - 28s 605ms/step - loss: 1.6639 - accuracy: 0.4497 - val_loss: 1.6623 - val_accuracy: 0.4591
Epoch 6/25
47/47 [==============================] - 28s 597ms/step - loss: 1.5480 - accuracy: 0.4872 - val_loss: 1.6187 - val_accuracy: 0.4638
Epoch 7/25
47/47 [==============================] - 27s 589ms/step - loss: 1.5111 - accuracy: 0.5034 - val_loss: 1.4864 - val_accuracy: 0.5067

In [11]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Path to the new image
image_path = "dataset\hssqb.jpeg" # Replace with the actual path

# Load and preprocess the new image
new_image = image.load_img(image_path, target_size=(64, 64))
new_image_array = image.img_to_array(new_image)
new_image_array = np.expand_dims(new_image_array, axis=0)
new_image_array /= 255.0  # Rescale to match the training data normalization

# Make a prediction
prediction = model.predict(new_image_array)

# Convert the prediction to a human-readable label
class_labels = ['Alt_cucu','Alt_b','Aphid','ArmW','Bact_LS','Bact_wilt','Cucum_beet','Flee_beet','Fusa','Gum_stem','Phy_bli','Sqau_bg','Thirps','Hlty_corn']
# internal mapping classes ['Alt_cucu','Alt_b','Aphid','ArmW','Bact_LS','Bact_wilt','Cucum_beet','Flee_beet','Fusa','Gum_stem','Phy_bli','Sqau_bg','Thirps','Hlty_corn']
# og file struct ['Bacspot','Eblight','Hlty','LateB','LeafM','septLeaf','SpidM','TarSpot','ToMV','YellowLeaf']
predicted_class_index = np.argmax(prediction)
predicted_class = class_labels[predicted_class_index]

# Display the results
print(f"The predicted class is: {predicted_class}")
print(f"Confidence scores: {prediction.squeeze()}")

1/1 [==============================] - 0s 58ms/step
The predicted class is: Sqau_bg
Confidence scores: [8.6449772e-06 1.1814810e-04 2.0114130e-07 2.6382944e-03 1.9561519e-08
 2.6475618e-06 4.5956092e-04 1.3495557e-01 8.2741881e-06 1.0024390e-05
 3.6413212e-06 8.6059904e-01 7.0736824e-05 1.1249463e-03 2.5987825e-07]


In [ ]:
#Alternaria cucumerina': 0, 'Alternaria leaf blight': 1, 'Aphids': 2, 'Armyworms': 3, 'Bacterial leaf spot': 4, 'Bacterial wilt': 5, 'Cucumber beetles': 6, 'Flea beetles': 7, 'Fusarium': 8, 'Gummy stem blight': 9, 'Phytophthora blight': 10, 'Squash bug': 11, 'Squash vine borer': 12, 'Thrips (Western flower thrips)': 13, 'healthy_pumpkin': 14}

In [9]:
accuracy = model.evaluate(test_set)[1]
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

47/47 [==============================] - 14s 284ms/step - loss: 0.5453 - accuracy: 0.8141
Accuracy on the test set: 81.41%


In [12]:
model.save('pumpkin_model.keras')